In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import sys

sys.path.append("../")
from src import soporte as sp
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

Parmetro de entrada, numero de paginas que queremos descargarnos

In [2]:
n_paginas_a_buscar = 100

In [3]:
def load_data_web(n_paginas_a_buscar_elementos):
    """
    Carga los datos de productos de un sitio web y los organiza en un DataFrame de pandas.

    La función realiza la extracción de datos de productos desde varias páginas web especificadas.
    Por cada página, realiza una solicitud HTTP, analiza el contenido HTML y extrae la información
    de los productos, incluyendo el nombre, categoría, sección, descripción, dimensiones e imagen.
    Los datos extraídos se concatenan en un único DataFrame.

    Args:
        n_paginas_a_buscar_elementos (int): El número de páginas web a recorrer para buscar productos.

    Returns:
        pandas.DataFrame: Un DataFrame que contiene la información de todos los productos extraídos,
                          con columnas que incluyen nombre, categoría, sección, descripción, dimensiones e imagen.

    Proceso:
        1. Inicializa un DataFrame vacío para almacenar los datos extraídos.
        2. Itera sobre el rango de páginas especificado para construir la URL y realizar una solicitud HTTP.
        3. Analiza el contenido HTML utilizando BeautifulSoup para encontrar los productos.
        4. Verifica si la lista de productos está vacía o si la solicitud falla (código de estado distinto de 200).
           En caso de error, interrumpe el proceso.
        5. Extrae los datos relevantes de cada producto y los almacena en un DataFrame.
        6. Concatenación de los DataFrames de cada producto en un DataFrame acumulativo.
        7. Devuelve el DataFrame final con todos los datos consolidados.

    Excepciones:
        - La función se interrumpe si no se encuentran productos en una página o si la solicitud HTTP falla,
          mostrando un mensaje de error con el código de estado.
"""
    df_all = pd.DataFrame()

    for pagina in range(1, (n_paginas_a_buscar_elementos + 1)):

        url = f"https://atrezzovazquez.es/shop.php?search_type=-1&search_terms=&limit=48&page={pagina}"
        url_imagen = "https://atrezzovazquez.es/"
        url_no_imagen = "https://atrezzovazquez.es/img/logo.png"
        res = requests.get(url)
        
        sopa = BeautifulSoup(res.content, "html.parser")
        lista_productos = sopa.findAll("div", {"class": "col-md-3 col-sm-4 shop-grid-item"} )

        if len(lista_productos) == 0 or res.status_code != 200:
                print(f"codigo de error: {res.status_code} o lista de productos vacios.")
                break
        else:

            elementos = sopa.findAll("div", {"class": "col-md-3 col-sm-4 shop-grid-item"})

            for e in elementos:
                ele = BeautifulSoup(str(e), "html.parser")
                
                nombre = sp.get_data_elemento_text(ele, 'a', 'class', 'title')
                categoria = sp.get_data_elemento_text(ele, 'a', 'class', 'tag')
                seccion = sp.get_data_elemento_text(ele, 'div', 'class', 'cat-sec')
                descripcion = sp.get_data_elemento_text(ele, 'div', 'class', 'article-container style-1')
                dimensiones = sp.get_data_elemento_text(ele, 'div', 'class', 'price')
                imagen = sp.get_data_elemento_imagen(ele, 'src', url_imagen, url_no_imagen)

                df_producto = sp.create_dataframe(nombre, categoria, seccion, descripcion, dimensiones, imagen)

                df_all = pd.concat([df_all, df_producto])
                df_all.reset_index(drop=True, inplace=True)
        
    return df_all

Guardamos el dataframe resultante

In [4]:
df_result = load_data_web(n_paginas_a_buscar)

Imprimimos el numero de registros que tiene

In [5]:
df_result.shape[0]

4800

Mostramos algunos

In [6]:
df_result.sample(3)

,nombre,categoria,seccion,descripcion,dimensiones,imagen
3629,TYV43,Televisores y vídeos,Televisor,"Televisión caja verde pistacho Electronic 24""",53x33x52 (cm),https://atrezzovazquez.es/admin/img_prod/TYV43...
341,JUE3,Juego,Casino,Mesa de Black Jack patas torneadas,158x84x77 (cm),https://atrezzovazquez.es/admin/img_prod/JUE3/...
1997,CHI19,Chimenea,None,Pantalla de chimenea forja hierro calada 3 hojas.,110x2x69 (cm),https://atrezzovazquez.es/admin/img_prod/CHI19...


Salvamos los datos a un csv, para su posterior tratamiento.

In [7]:
df_result.to_csv("../datos/listado_productos.csv")